# Notebook for development and testing

In [1]:
from torch.utils.data import DataLoader
from DataLoader import ship_dataset, init_dataloader
from Models import CNN_REG
from TrainingModel import ship_training_app
from collections import namedtuple
from multiprocessing import Process

def process_function(train_dl, valid_dl, model_params):
    training = ship_training_app(dl_train, dl_valid, model_params)
    training.main()


data_params = namedtuple(
    'data_params',
    'root_dir, dataset, use_gpu, num_workers, batch_size',
)

model_params = namedtuple(
    'model_params',
    'name, opt_name, gpu, aug_model_name, epochs, valid_epochs, max_pooling_rate, scaler, learning_rate',
)

model_params_att = namedtuple(
    'model_params_att',
    'name, opt_name, gpu, aug_model_name, epochs, valid_epochs, blocks, heads, emb_scale, num_emb, heads_shape, learning_rate',
)

model_params_att_train = model_params_att("ATT_NN", 'ADAM', True, 'None', 
                                            250, 2, 3, 4, 700, 32, [512, 512, 256, 128, 1], 0.0005)

data_params_train = data_params("/home/franko/Desktop/DSelimovcProject/Dataset/", 'train', True, 2 ,32)
data_params_valid = data_params("/home/franko/Desktop/DSelimovcProject/Dataset/", 'valid', True, 1 ,32)
dl_train = init_dataloader(data_params_train)
dl_valid = init_dataloader(data_params_valid)


p = Process(target = process_function, args = (dl_train, dl_valid, model_params_att_train))
p.start()
p.join()

gpu = True
epochs = 250
valid_epochs = 2
for model_name in ['CNN_REG', 'MLSTM_CNN', 'SP_NN']:
    for opt in ['SGD', 'ADAM']: 
        for aug in ['None', 'row-wise', 'batch-wise']:
            for max_pooling_rate in [1, 5, 10]:
                for scaler in [1, 5, 10]:
                    for learning_rate in [0.005, 0.0005, 0.00005]:
                        model_params_train = model_params(model_name, opt, gpu, aug, 
                                                   epochs, valid_epochs, max_pooling_rate, scaler,
                                                   learning_rate)
                        p = Process(target = process_function, args =(dl_train, dl_valid, model_params_train))
                        p.start()
                        p.join()
    
# model_params = model_params('CNN_REG', 'ADAM', True, 500, 5, 10, 10, 0.0005 ) OVO JE BILO ISPOD 0.00868 135 epochs

**************************************
USING MODEL: ATT_NN
USING GPU: cuda
USING OPTIMIZER: ADAM / LR:0.0005
USING AUG TYPE: None
Starting training!
**************************************
Epoch 1 / 250
torch.Size([32, 32, 1, 802])


Process Process-1:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_20050/2694027208.py", line 10, in process_function
    training.main()
  File "/home/franko/Desktop/DSelimovcProject/Scripts/TrainingModel.py", line 386, in main
    _metrics = self.train_model(self.train_dl)
  File "/home/franko/Desktop/DSelimovcProject/Scripts/TrainingModel.py", line 130, in train_model
    _loss.backward()
  File "/home/franko/pytorch/lib/python3.7/site-packages/torch/tensor.py", line 221, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "/home/franko/pytorch/lib/python3.7/site-packages/torch/autograd/__init__.py", line 132, in backward
    allow_unreachable=True)  # allow_unreachable flag
RuntimeError: CUDA out of memory. Tried to allocate

**************************************
USING MODEL: CNN_REG
USING GPU: cuda
USING OPTIMIZER: SGD / LR:0.005
USING AUG TYPE: None
Starting training!
**************************************
Epoch 1 / 250
Train: 1 : MSE:0.02713, MAE:0.12768
Saving model!
*************
Valid: 1 : MSE:0.01813, MAE:0.10221
*************
Saving model!
Saving best model!
Epoch 2 / 250
Train: 2 : MSE:0.01603, MAE:0.09848
Saving model!
*************
Valid: 2 : MSE:0.01779, MAE:0.10260
*************
Saving model!
Saving best model!
Epoch 3 / 250
Train: 3 : MSE:0.01405, MAE:0.09141
Saving model!
Epoch 4 / 250
Train: 4 : MSE:0.01286, MAE:0.08688
Saving model!
*************
Valid: 4 : MSE:0.01833, MAE:0.10218
*************
Epoch 5 / 250
Train: 5 : MSE:0.01198, MAE:0.08305
Saving model!
Epoch 6 / 250
Train: 6 : MSE:0.01122, MAE:0.08008
Saving model!
*************
Valid: 6 : MSE:0.01900, MAE:0.10297
*************
Epoch 7 / 250
Train: 7 : MSE:0.01054, MAE:0.07712
Saving model!
Epoch 8 / 250
Train: 8 : MSE:0.00999, MAE:0

Process Process-2:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_20050/2694027208.py", line 10, in process_function
    training.main()
  File "/home/franko/Desktop/DSelimovcProject/Scripts/TrainingModel.py", line 386, in main
    _metrics = self.train_model(self.train_dl)
  File "/home/franko/Desktop/DSelimovcProject/Scripts/TrainingModel.py", line 127, in train_model
    _loss = self.get_loss(_index, _batch, _metrics)
  File "/home/franko/Desktop/DSelimovcProject/Scripts/TrainingModel.py", line 191, in get_loss
    _prediction = self.model(_input_data)
  File "/home/franko/pytorch/lib/python3.7/site-packages/torch/nn/modules/module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/home/franko/Desktop/DSelimovcProject/Scripts/Models

KeyboardInterrupt: 